# Gesundheitsausgaben-Analyse

## Projektübersicht
Dieses Projekt analysiert Gesundheitsausgaben basierend auf verschiedenen Kategorien wie Finanzierungsquellen, Arten von Gesundheitsleistungen und Zeiträumen.

Ziel ist es, Einblicke in die Entwicklung der Gesundheitskosten zu gewinnen und Muster zu identifizieren, die zukünftige Entscheidungen in der Gesundheitspolitik beeinflussen können.

### Projektziele:
1. **Datenbereinigung:** Aufbereitung der Rohdaten durch Entfernen unnötiger Spalten.
2. **Datenanalyse:** Identifikation von Trends in den Gesundheitsausgaben.
3. **Visualisierung:** Erstellung von Diagrammen für die Interpretation.
4. **Berichterstellung:** Zusammenfassung der wichtigsten Erkenntnisse.

### Verwendete Datensätze:
Die Analyse basiert auf folgenden CSV-Dateien:
- `OGD_gesausgaben01_HVD_HCHF_1_C-HCGES-0.csv` (Gesundheitsleistungen und -güter)
- `OGD_gesausgaben01_HVD_HCHF_1_C-ZEITGES-0.csv` (Zeitreihen der Gesundheitsausgaben)
- `OGD_gesausgaben01_HVD_HCHF_1_HEADER.csv` (Metadaten über Spaltenbezeichnungen)

### Projektstruktur:
- `data/` → Enthält die Anfangsdaten
- `notebooks/` → Enthält das Jupyter Notebook
- `output/` → Speichert bereinigte und aggregierte Daten
- `scripts/` → Enthält wiederverwendbare Python-Funktionen

## Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:

# Cleanup (helps with debugging)
import subprocess


for dirname in "data/corr", "data/prep":
    if os.path.exists(dirname):
        subprocess.run(["rm", "-rf", dirname])
    os.makedirs(dirname)


## 1. Datenanalyse

1. **Hauptdatensatz:**  
   - Datei: `OGD_gesausgaben01_HVD_HCHF_1.csv`  
   - Enthält die eigentlichen Gesundheitsausgaben-Daten.  

2. **Hilfsdatensätze:**  
   - `OGD_gesausgaben01_HVD_HCHF_1_C-HCGES-0.csv` – Gesundheitsleistungen und -güter (HC)  
   - `OGD_gesausgaben01_HVD_HCHF_1_C-ZEITGES-0.csv` – Zeitreihen der Gesundheitsausgaben  
   - `OGD_gesausgaben01_HVD_HCHF_1_HEADER.csv` – Header-Metadaten zur Beschreibung der Spalten  

#### Warum verwenden wir `;` als Delimiter?  

Die bereitgestellten CSV-Dateien verwenden das Semikolon (`;`) als Trennzeichen (wir glauben um Verwechslungen mit Dezimalpunkten oder -kommas zu vermeiden)

Daher muss beim Einlesen `delimiter=';'` explizit angegeben werden, um die Daten korrekt zu parsen.

Später im Projekt werden wir Beistrich (`','`) als Delimiter verwenden, um die Daten zu speichern, da dies der Standard in den meisten CSV-Dateien ist.


In [ ]:
def analyze_dataframe(df, name):
    print(f"\n--- {name} ---")

    print("\nErste 5 Zeilen:")
    display(df.head())

    print("\nZufällige Stichprobe:")
    display(df.sample(5))

    print("\nDateninfo:")
    df.info()

    print("\nStatistiken (numerische und kategoriale Werte):")
    display(df.describe())

    # Unique Werte für jede Spalte (nur für kategorische Spalten sinnvoll)
    print("\nAnzahl eindeutiger Werte pro Spalte:")
    for col in df.columns:
        unique_values = df[col].nunique()
        print(f"{col}: {unique_values} eindeutige Werte")
        # print(f"  Werte: {df[col].unique()}")  # unnötig zu viel output

    # ganz wichtig noch null werte!
    print(f"Fehlende Werte in {name}:")
    display(df.isnull().sum())

In [ ]:
dataframes = {}
for filename in os.listdir("data"):
    if not filename.endswith('.csv') and not filename.startswith('OGD_gesausgaben01_HVD_HCHF'): # SCHEISS auf die anderen dateien
        continue
    df_ = pd.read_csv(os.path.join("data", filename), delimiter=';')
    df_name = filename.replace('.csv', '')
    dataframes[df_name] = df_

df_main = dataframes['OGD_gesausgaben01_HVD_HCHF_1']
df_hc = dataframes['OGD_gesausgaben01_HVD_HCHF_1_C-HCGES-0']
df_time = dataframes['OGD_gesausgaben01_HVD_HCHF_1_C-ZEITGES-0']
df_header = dataframes['OGD_gesausgaben01_HVD_HCHF_1_HEADER']

for df_name, df in dataframes.items():
    analyze_dataframe(df, df_name)

### Ergebnisse:  

**1. Hauptdatensatz (Gesundheitsausgaben)**  
- **Anzahl:** 729 Zeilen, 16 Spalten.  
- **Datentypen:** Alle Spalten sind vom Typ `object`. Dies deutet darauf hin, dass möglicherweise numerische Werte als Text gespeichert wurden und eine Umwandlung notwendig sein könnte.  
- **Fehlende Werte:** Keine fehlenden Werte festgestellt.  
- **Mögliche Maßnahmen:** Überprüfung der Datenformate (z. B. Zahlen als Text) und mögliche Konvertierung in numerische Typen.  

---

**2. Gesundheitsleistungen und -güter (HC)**  
- **Anzahl:** 46 Zeilen, 10 Spalten.  
- **Fehlende Werte:**  
  - Spalten `de_desc`, `de_link`, `en_desc`, `en_link`, `de_syn`, `en_syn` enthalten ausschließlich fehlende Werte und sollten entfernt werden.  
  - Die Spalte `FK` weist 7 fehlende Werte auf, welche genauer überprüft werden müssen (z. B. Kategorie oder Obergruppe).  
- **Mögliche Maßnahmen:**  
  - Löschen irrelevanter Spalten mit ausschließlich fehlenden Werten.  
  - Umgang mit fehlenden Werten in `FK` (Ersatz, Gruppierung oder Löschung).  

---

**3. Zeitreihen der Gesundheitsausgaben**  
- **Anzahl:** 19 Zeilen, 10 Spalten.  
- **Fehlende Werte:**  
  - Die Spalte `Unnamed: 2` enthält nur `NaN`-Werte und sollte entfernt werden.  
  - Weitere Spalten (`de_desc`, `de_link`, etc.) sind ebenfalls vollständig leer.  
- **Datentypen:**  
  - Die Spalten `name` und `en_name` sind als `int64` klassifiziert – sollte überprüft werden, ob dies korrekt ist oder in `string`/`category` geändert werden muss.  
- **Mögliche Maßnahmen:**  
  - Entfernen leerer Spalten.  
  - Datentypen anpassen, falls erforderlich.  

---

**4. Header-Metadaten**  
- **Anzahl:** 16 Zeilen, 10 Spalten.  
- **Fehlende Werte:**  
  - Unbenannte Spalten `Unnamed: 3` bis `Unnamed: 9` enthalten ausschließlich fehlende Werte und können entfernt werden.  
- **Mögliche Maßnahmen:**  
  - Entfernen leerer Spalten zur Reduzierung unnötiger Daten.

### Konvertierung der Daten in numerische Werte

In [ ]:
# TWO DAYS AND 10 CHIPS PACKETS AND 50 COFFES LATER WE DISCOVERED THAT THE DATA IS NOT NUMERIC AND WE NEED TO REPLACE , WITH . AND CONVERT TO NUMERIC
# DHIS BULLSHIT IS NOT NUMERIC
# df_main.iloc[:, 2:] = df_main.iloc[:, 2:].replace(',', '.', regex=True).apply(pd.to_numeric)
# THIS IS NOT WORKING
# WE JUST SAVE THE DATAFRAME TO CSV AND READ IT AGAIN

# F*ck it, we save the data to csv and read it again

df_main.replace(',', '.', regex=True).to_csv('data/corr/OGD_gesausgaben01_HVD_HCHF_1_corr.csv', index=False)  # replace , with . for numeric conversion
df_main = pd.read_csv('data/corr/OGD_gesausgaben01_HVD_HCHF_1_corr.csv') # read and check if it worked
df_main.info()

### Bereinigung der Sprach- und Null-Spalten

In den Datensätzen sind Spalten in deutscher und englischer Sprache enthalten.  
Da die englischen Spalten redundant sind, werden sie entfernt.  
Zusätzlich werden alle vollständig leeren Spalten aus den Daten gelöscht.


In [ ]:
def clean_df(dataframe, name):
    print(f"\n--- Bereinigung für {name} ---")

    # 1. Englische Spalten entfernen (beginnen mit 'en_')
    df_cleaned = df.drop(columns=[col for col in df.columns if col.startswith('en_')])

    # 2. Vollständig leerer Spalten
    empty_cols = df_cleaned.columns[df_cleaned.isnull().all()].tolist()
    print(f"Leere Spalten in {name}: {empty_cols}")

    # 3. Entfernen ovn leeren Spalten
    df_cleaned.drop(columns=empty_cols, inplace=True)

    # 4. Shape (Rows, Cols) nach Bereinigung
    print(f"Nach Bereinigung der englischen und leeren Spalten - {name}:")
    print(f"{name} shape:", df_cleaned.shape)

    # 5. Speichern der bereinigten DataFrames
    file_name = name + "_corr.csv"
    df_cleaned.to_csv(f'data/corr/{file_name}', index=False)
    print(f"Bereinigte Datei für {name} gespeichert als {file_name}")

del dataframes['OGD_gesausgaben01_HVD_HCHF_1']  # remove main dataframe (already cleaned)
for name, df in dataframes.items():
    clean_df(df, name)

### Ersetzung von Codes durch Namen

Nach der ersten Datenbereinigung werden die Codes in den Spalten `C-ZEITGES-0` und `C-HCGES-0`  
durch verständliche Namen ersetzt. Dies geschieht mit den Mapping-Tabellen aus den  
Header- und Gesundheitsleistungsdateien.

Bevor das getan werden kann, mappen wir die Oberkategorien in der HCGES Datei auf die Unterkategorien. 

Nach der Ersetzung speichern wir die aktualisierten Daten für die weitere Verarbeitung. Außerdem löschen wir die unnötigen Files.


In [ ]:
df_main_corr = df_main # copy the main dataframe, don't need to read it again
df_hc_corr = pd.read_csv('data/corr/OGD_gesausgaben01_HVD_HCHF_1_C-HCGES-0_corr.csv')
df_time_corr = pd.read_csv('data/corr/OGD_gesausgaben01_HVD_HCHF_1_C-ZEITGES-0_corr.csv')
df_header_corr = pd.read_csv('data/corr/OGD_gesausgaben01_HVD_HCHF_1_HEADER_corr.csv')

In [ ]:

# --- 1. Mapping für 'C-HCGES-0' mit df_hc_corr ---
hc_mapping = df_hc_corr.set_index('code')['name'].to_dict()
df_main_corr['C-HCGES-0'] = df_main_corr['C-HCGES-0'].apply(lambda x: hc_mapping.get(x, x))

# --- 2. Mapping für 'C-ZEITGES-0' mit df_time_corr ---
time_mapping = df_time_corr.set_index('code')['name'].to_dict()
df_main_corr['C-ZEITGES-0'] = df_main_corr['C-ZEITGES-0'].apply(lambda x: time_mapping.get(x, x))

# --- 3. Mapping der Spaltennamen mit df_header_corr ---
header_mapping = df_header_corr.set_index('code')['name'].to_dict()
df_main_corr.rename(columns=header_mapping, inplace=True)

df_main_corr.to_csv('data/corr/OGD_gesausgaben01_HVD_HCHF_1_corr.csv', index=False)

### Datenvorbereitung und Speicherung

Nach der Vorverarbeitung der Daten werden die bereinigten Daten in verschiedene Kategorien unterteilt und separat gespeichert.  

##### 1. Numerische Werte  
- Enthält alle Spalten mit numerischen Daten (z. B. `int64`, `float64`).  
- Diese Daten werden extrahiert und in einer separaten Datei gespeichert.  
- **Dateiname:** `num.csv`  

##### 2. Nominale Werte  
- Enthält alle kategorialen und textbasierten Daten (z. B. `category`, `object`).  
- Diese Werte werden extrahiert und separat gespeichert.  
- **Dateiname:** `nom.csv`  

##### 3. Normalisierte Werte  
- Für eine bessere Vergleichbarkeit und Skalierung werden die numerischen Daten normalisiert.  
- Die Min-Max-Normalisierung wird verwendet, um die Werte in den Bereich von 0 bis 1 zu bringen.  
- **Dateiname:** `norm.csv`


In [ ]:
# Numerische Werte extrahieren
df_main_corr.info()
df_numeric = df_main_corr.select_dtypes(include=['int64', 'float64'])
df_numeric.to_csv('data/prep/num.csv', index=False)

# Nominale Werte extrahieren (kategorische und objektbasierte Spalten)
df_nominal = df_main_corr.select_dtypes(include=['category', 'object'])
df_nominal.to_csv('data/prep/nom.csv', index=False)

# Normalisierung der numerischen Werte (Min-Max-Normalisierung)
df_normalized = (df_numeric - df_numeric.min()) / (df_numeric.max() - df_numeric.min())
df_normalized.to_csv('data/prep/norm.csv', index=False)